Create a DataFrame from an RDD through reflection method

Import findspark and initiate.
Then import pyspark

In [1]:
import findspark
findspark.init()
import pyspark

Start SparkSession

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Python Spark SQL example").getOrCreate()

In [3]:
from pyspark.sql import Row

In [4]:
sc = spark.sparkContext

Create an RDD from the structured text file

In [5]:
clines = sc.textFile("customers.tsv")

Transform this RDD of text lines to an RDD of "Row"s with each Row having the 5 fields of the text input lines.

In [6]:
cfields = clines.map(lambda l: l.split("\t"))

In [7]:
customers = cfields.map(lambda p: Row(cid=int(p[0]),cname=p[1],ccity=p[2],cstate=p[3],czip=p[4]))

Infer the schema, and register the DataFrame as a table (temp view).

In [8]:
customerDF = spark.createDataFrame(customers)

In [9]:
customerDF.printSchema()

root
 |-- ccity: string (nullable = true)
 |-- cid: long (nullable = true)
 |-- cname: string (nullable = true)
 |-- cstate: string (nullable = true)
 |-- czip: string (nullable = true)



In [10]:
customerDF.select("cname").show()

+----------------+
|           cname|
+----------------+
|     Mary Torres|
|      Jose Haley|
|      Mary Smith|
|  Richard Maddox|
|  Margaret Booth|
|  Mary Henderson|
|     Lisa Walker|
|   Jonathan Hill|
|Carolyn Sheppard|
|    Mary Mendoza|
|   Michael Smith|
|    James Holmes|
|     Mary Dawson|
|    Adam Marquez|
|    Gloria Smith|
|       Mary Webb|
|  Nancy Alvarado|
|  Russell Flores|
|    Denise Smith|
|  Jose Dickerson|
+----------------+
only showing top 20 rows



In [11]:
customerDF.select(customerDF['cname'], customerDF['ccity']).show()

+----------------+-------------+
|           cname|        ccity|
+----------------+-------------+
|     Mary Torres|       Caguas|
|      Jose Haley|     Columbus|
|      Mary Smith|      Houston|
|  Richard Maddox|       Caguas|
|  Margaret Booth|    Arlington|
|  Mary Henderson|       Caguas|
|     Lisa Walker|       Caguas|
|   Jonathan Hill|      Phoenix|
|Carolyn Sheppard|Pompano Beach|
|    Mary Mendoza|       Caguas|
|   Michael Smith|       Caguas|
|    James Holmes|     Hilliard|
|     Mary Dawson|       Caguas|
|    Adam Marquez|  San Antonio|
|    Gloria Smith|       Caguas|
|       Mary Webb|   San Marcos|
|  Nancy Alvarado|     Flushing|
|  Russell Flores|       Caguas|
|    Denise Smith|    Rego Park|
|  Jose Dickerson|         Mesa|
+----------------+-------------+
only showing top 20 rows



In [12]:
customerDF.filter(customerDF['cstate'] == 'CA').show()

+---------------+-----+----------------+------+-----+
|          ccity|  cid|           cname|cstate| czip|
+---------------+-----+----------------+------+-----+
|        Modesto| 5577|      Mary Smith|    CA|95350|
|Rowland Heights| 1745|      Mary Smith|    CA|91748|
|      San Diego|11444|Kathleen Patrick|    CA|92109|
|          Indio| 8846|    Thomas Smith|    CA|92201|
|       El Cajon| 6237|  Bobby Anderson|    CA|92020|
|  Panorama City| 4085|       Mary Carr|    CA|91402|
|       Stockton| 8705|  Patricia Smith|    CA|95207|
| San Bernardino| 3669|       Mary Soto|    CA|92410|
|    Los Angeles| 6101|      Mary Smith|    CA|90033|
|  Laguna Niguel|11697|  Jessica Thomas|    CA|92677|
|       Winnetka| 1295|   Theresa Lopez|    CA|91306|
|    Simi Valley| 4814|     Paul Suarez|    CA|93065|
|       Highland| 8530|   William Smith|    CA|92346|
|        Ontario| 3846|    Ronald Lewis|    CA|91764|
|       Cerritos|10476|     John Hodges|    CA|90703|
|    Los Angeles|10243|  Don

In [13]:
customerDF.groupBy("cstate").count().show()

+------+-----+
|cstate|count|
+------+-----+
|    AZ|   19|
|    SC|    2|
|    LA|    7|
|    MN|    1|
|    NJ|   19|
|    DC|    4|
|    OR|    4|
|    VA|   14|
|    RI|    2|
|    KY|    1|
|    MI|   28|
|    NV|   16|
|    WI|    9|
|    ID|    2|
|    CA|  187|
|    CT|    8|
|    NC|   19|
|    MD|   19|
|    DE|    1|
|    MO|   13|
+------+-----+
only showing top 20 rows



Create the temp view to be able to run SQL queries on the DataFrame

In [14]:
customerDF.createOrReplaceTempView("customers")

In [15]:
cStateCount50 = spark.sql("SELECT cstate, count(*) as sttcount FROM customers GROUP BY cstate HAVING sttcount>=50")

In [16]:
cStateCount50.show()

+------+--------+
|cstate|sttcount|
+------+--------+
|    CA|     187|
|    NY|      79|
|    TX|      62|
|    PR|     505|
+------+--------+



In [17]:
cStateCount50.printSchema()

root
 |-- cstate: string (nullable = true)
 |-- sttcount: long (nullable = false)



In [18]:
type(cStateCount50)

pyspark.sql.dataframe.DataFrame

In [19]:
sc.stop()